<a href="https://colab.research.google.com/github/Drime648/tor-deep-learning/blob/main/Tor_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports

import pandas as pd
import zipfile
import os, os.path
import pathlib
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [ ]:
!nvidia-smi

Tue Jul 26 14:42:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Extracting the data

We extract the data, split it, and put it into train and test dataframes

In [ ]:
!wget https://github.com/kdsec/wangknn-dataset/raw/master/data.zip
import zipfile
zipref = zipfile.ZipFile("data.zip")
zipref.extractall()
# zipref.close()
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

unzip_data("data.zip")
walk_through_dir("/content/data.zip")

--2022-07-26 14:42:53--  https://github.com/kdsec/wangknn-dataset/raw/master/data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kdsec/wangknn-dataset/master/data.zip [following]
--2022-07-26 14:42:54--  https://raw.githubusercontent.com/kdsec/wangknn-dataset/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77101936 (74M) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>]  73.53M   167MB/s    in 0.4s    

2022-07-26 14:42:54 (167 MB/s) - ‘data.zip.1’ saved [77101936/77101936]

--2022-07-26 14:43:08--  https://raw.githubusercontent.com/mr

In [ ]:
# walk_through_dir("/content")

In [ ]:
# !ls

In [ ]:
# import pandas as pd

train_df = pd.DataFrame(columns = ["page_num", "trace_num", "trace_data"])
test_df = pd.DataFrame(columns = ["page_num", "trace_num", "trace_data"])
# #72-18 split

#page_num goes from 0-99, this is the classes we will predict
#trace_num goes from 0-89, as each monitored site has 90 traces.
#In total it is 90*100 = 9,000 traces

In [ ]:
import os, os.path
import pathlib

PATH = pathlib.Path("/content")
path, dirs, files = next(os.walk(PATH))

In [ ]:
def get_lines(filename):
  with open(filename) as f:
    return f.readlines()

In [ ]:
def preprocess_lines(filename):
  input_lines = get_lines(filename)
  samples = []

  for line in input_lines:
    x = line.split()
    samples.append(int(x[1]))

    
  return samples

In [ ]:
page_num = 0
trace_num = 0
trace_data = []
for file in files:
  if("-" in file):
    # print(file)
    x = file.split("-")
    page_num = int(x[0])
    trace_num = int(x[1])
    trace_data = preprocess_lines(file)
    if(len(trace_data) > 0):#Filter can be anything. Right now it is zero
    
      #make sure the length is exactly 5000
      while(len(trace_data) < 5000):
          trace_data.append(0)
      
      trace_data = trace_data[:5000]
      # print(len(trace_data))
      # append everything to the dataframe
      temp_df = {"page_num": page_num, "trace_num": trace_num, "trace_data": trace_data}

      #first, decide if it is test or train
      #72-18 split
      #trace nums go from 0-89
      #0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17 - trace nums for the test dataset
      if(trace_num >= 17): # add to the train dataset
        train_df = train_df.append(temp_df, ignore_index=True)
      else:
        test_df = test_df.append(temp_df, ignore_index=True)


Making a graph to see the ranges of the lengths of the trace data

In [ ]:
# page_num = 0
# trace_num = 0
# trace_data = []
# for file in files:
#   if("-" in file):
#     # print(file)
#     x = file.split("-")
#     page_num = int(x[0])
#     trace_num = int(x[1])
#     trace_data = preprocess_lines(file)
#     if(len(trace_data) > 0):#in case it is an outlier
    
#       #make sure the length is exactly 5000
#       # while(len(trace_data) < 5000):
#       #     trace_data.append(0)
      
#       # trace_data = trace_data[:5000]
#       # print(len(trace_data))
#       # append everything to the dataframe
#       temp_df = {"page_num": page_num, "trace_num": trace_num, "trace_data": trace_data}

#       #first, decide if it is test or train
#       #72-18 split
#       #trace nums go from 0-89
#       #0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17 - trace nums for the test dataset
#       train_df = train_df.append(temp_df, ignore_index=True)
#       # if(trace_num >= 17): # add to the train dataset
#       #   train_df = train_df.append(temp_df, ignore_index=True)
#       # else:
#       #   test_df = test_df.append(temp_df, ignore_index=True)

In [ ]:
# train_df

In [ ]:
# count = 0
# num_array = []
# for index, row in train_df.iterrows():
#   num = len(row["trace_data"])
#   num_array.append(num)




In [ ]:
# from matplotlib import pyplot as plt
# import numpy as np

# plt.hist(num_array, bins = np.linspace(0, 5000, 30))

In [ ]:
train_df

,page_num,trace_num,trace_data
0,65,88,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
1,26,89,"[1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1..."
2,30,67,"[-1, -1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,15,55,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
4,17,31,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
...,...,...,...
7295,59,80,"[-1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1..."
7296,63,51,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."
7297,17,30,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
7298,11,55,"[1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [ ]:
test_df

,page_num,trace_num,trace_data
0,64,0,"[1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, ..."
1,62,15,"[1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,47,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,30,7,"[1, 1, 1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1,..."
4,74,6,"[-1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, -..."
...,...,...,...
1695,99,10,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."
1696,62,4,"[1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1697,56,15,"[1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1,..."
1698,34,0,"[1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1,..."


In [ ]:
# train_df.to_csv("tor_dl_training_nofilter.csv")
# test_df.to_csv("tor_dl_testing_nofilter.csv")

In [ ]:
# from google.colab import files
# files.download("tor_dl_training_nofilter.csv") 
# files.download("tor_dl_testing_nofilter.csv")

In [ ]:
# !ls | grep "tor"

#Import the cleaned data now

In [ ]:
# train_df = pd.read_csv("https://raw.githubusercontent.com/Drime648/tor-deep-learning/main/tor_dl_training.csv")
# test_df = pd.read_csv("https://raw.githubusercontent.com/Drime648/tor-deep-learning/main/tor_dl_testing.csv")

In [ ]:
#remove the index column
# train_df = train_df.drop("Unnamed: 0", axis = 1)
# test_df = test_df.drop("Unnamed: 0", axis = 1)



In [ ]:
#randomize
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df

,page_num,trace_num,trace_data
0,57,77,"[-1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1,..."
1,50,41,"[1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1..."
2,97,30,"[1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1..."
3,51,38,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -..."
4,42,88,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
...,...,...,...
7295,69,59,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7296,69,85,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7297,5,52,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7298,51,26,"[1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1..."


In [ ]:
test_df

,page_num,trace_num,trace_data
0,53,10,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1,..."
1,68,7,"[1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -..."
2,88,7,"[1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1,..."
3,99,15,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."
4,30,9,"[-1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1..."
...,...,...,...
1695,39,5,"[-1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -..."
1696,78,16,"[1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1..."
1697,96,8,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
1698,74,12,"[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1..."


# replicate original model from Paper

Paper: https://core.ac.uk/download/pdf/229876143.pdf

Fingerprinting Attack on Tor Anonymity using Deep Learning


by Kota Abe and Shigeki Goto



#One hot encode data


In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")

In [ ]:
# add one more to test data
# temp_df = {"page_num": 300, "trace_num": 0, "trace_data": 0}
# test_df = test_df.append(temp_df, ignore_index = True)

In [ ]:
one_hot_test_labels = one_hot_encoder.fit(test_df["page_num"].to_numpy().reshape(-1,1))
# test_df = test_df[:-1]

one_hot_test_labels = one_hot_encoder.transform(test_df["page_num"].to_numpy().reshape(-1,1))
one_hot_train_labels = one_hot_encoder.transform(train_df["page_num"].to_numpy().reshape(-1,1))




In [ ]:
test_df

,page_num,trace_num,trace_data
0,53,10,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1,..."
1,68,7,"[1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -..."
2,88,7,"[1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1,..."
3,99,15,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."
4,30,9,"[-1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1..."
...,...,...,...
1695,39,5,"[-1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -..."
1696,78,16,"[1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1..."
1697,96,8,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
1698,74,12,"[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1..."


In [ ]:
one_hot_test_labels.shape

(1700, 100)

In [ ]:
one_hot_train_labels.shape

(7300, 100)

#process data

In [ ]:
train_data = []

for index, row in train_df.iterrows():
  # strings = row["trace_data"][1:-1].split(", ")
  # # strings[0]
  # # print(strings)
  # int_array = [int(number) for number in strings]
    train_data.append(row["trace_data"])


In [ ]:
test_data = []

for index, row in test_df.iterrows():

  # strings = row["trace_data"][1:-1].split(", ")
  # # strings[0]
  # # print(strings)
  # int_array = [int(number) for number in strings]
  test_data.append(row["trace_data"])

In [ ]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [ ]:
np.shape(train_data)

(7300, 5000)

In [ ]:
train_df

,page_num,trace_num,trace_data
0,57,77,"[-1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1,..."
1,50,41,"[1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1..."
2,97,30,"[1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1..."
3,51,38,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -..."
4,42,88,"[1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ..."
...,...,...,...
7295,69,59,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7296,69,85,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7297,5,52,"[1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, ..."
7298,51,26,"[1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1..."


#Make the dataset

In [ ]:
BATCH_SIZE = 50

In [ ]:
train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, one_hot_train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, one_hot_test_labels))

In [ ]:
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_dataset, train_dataset

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 5000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100), dtype=tf.float64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 5000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100), dtype=tf.float64, name=None))>)

##SDAE + MLP

Make 2 layers of SDAE, then followed by MLP

5000 - 500 - 125 - 500 - 5000 - 100

SDAE is: 
5000 - 500 - 125 - 500 - 5000

lr = 0.001, batch size = 50


MLP is:

5000 -> 100




In [ ]:
# layer_1 = 5000
# layer_2 = 750
# layer_3 = 500
# center_layer = 250

# model_1 = tf.keras.Sequential([
#   tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
#   tf.keras.layers.Dense(layer_2, "relu", kernel_initializer="he_normal"),
#   tf.keras.layers.Dense(center_layer, "relu", kernel_initializer="he_normal"),
#   tf.keras.layers.Dense(layer_2, "relu", kernel_initializer="he_normal"),
#   tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
#   tf.keras.layers.Dense(100, "softmax", kernel_initializer="he_normal"),
# ])

In [ ]:
import os

def make_model_checkpoint(name, path = "/content/model_exp/"):
  return tf.keras.callbacks.ModelCheckpoint(os.path.join(path, name), save_best_only = True, verbose = 0, monitor="val_accuracy")
  #, monitor="val_accuracy"

In [ ]:
# model_1.compile(loss = "categorical_crossentropy",
#                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                 metrics = ["accuracy"],
#               )

# history_1 = model_1.fit(train_dataset, epochs = 100,
#                         validation_data=test_dataset, 
#                     callbacks = [make_model_checkpoint("model_1")])

In [ ]:
# new_model_1 = tf.keras.models.load_model("/content/model_exp/model_1")
# new_model_1.evaluate(test_dataset)

In [ ]:
# plt.figure(figsize=(9, 9))
# plt.plot(history_1.history['val_accuracy'])
# plt.title('2 layer SDAE close world testing accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['testing accuracy'], loc='upper left')


# plt.savefig('2 layer SDAE close world testing accuracy.png', bbox_inches='tight')
# plt.show()

# files.download('2 layer SDAE close world testing accuracy.png')

In [ ]:
!nvidia-smi

Tue Jul 26 14:44:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    29W /  70W |    796MiB / 15109MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##try triple layer

In [ ]:
# layer_1 = 5000
# layer_2 = 750
# layer_3 = 500
# center_layer = 250

# model_2 = tf.keras.Sequential([
#   tf.keras.layers.Dense(layer_1, "relu"),
#   tf.keras.layers.Dense(layer_2, "relu"),
#   tf.keras.layers.Dense(layer_3, "relu"),
#   tf.keras.layers.Dense(center_layer, "relu"),
#   tf.keras.layers.Dense(layer_3, "relu"),
#   tf.keras.layers.Dense(layer_2, "relu"),
#   tf.keras.layers.Dense(layer_1, "relu"),
#   tf.keras.layers.Dense(100, "softmax"),
# ])

In [ ]:
# model_2.compile(loss = "categorical_crossentropy",
#                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                 metrics = ["accuracy"],
#               )

# history_2 = model_2.fit(train_dataset, epochs = 100,
#                         validation_data=test_dataset, 
#                     callbacks = [make_model_checkpoint("model_2")])

In [ ]:
# new_model_2 = tf.keras.models.load_model("/content/model_exp/model_2")
# new_model_2.evaluate(test_dataset)

In [ ]:
# plt.figure(figsize=(9, 9))
# plt.plot(history_2.history['val_accuracy'])
# plt.title('3 layer SDAE close world testing accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['testing accuracy'], loc='upper left')


# plt.savefig('3 layer SDAE close world testing accuracy.png', bbox_inches='tight')
# plt.show()

# files.download('3 layer SDAE close world testing accuracy.png')

#Improve model

##model 3

In [ ]:
import tensorflow.keras.layers as layers

In [ ]:
# model_3 = tf.keras.Sequential([
#   layers.Lambda(lambda x: tf.expand_dims(x, axis = 1)),
#   layers.Conv1D(5000, 5, padding = 'causal', activation="relu"),
#   layers.GlobalMaxPooling1D(),
#   layers.Dense(100, activation="softmax")
# ], name = "Conv1D")

# model_3.compile(loss = "categorical_crossentropy",
#                 optimizer = "Adam",
#                 metrics = ["accuracy"])

# history_3 = model_3.fit(train_dataset, epochs = 40,
#                         validation_data=test_dataset, 
#                     callbacks = [make_model_checkpoint("model_3")])

In [ ]:
# new_model_3 = tf.keras.models.load_model("/content/model_exp/model_3")
# new_model_3.evaluate(test_dataset)

In [ ]:
# plt.figure(figsize=(9, 9))
# plt.plot(history_3.history['val_accuracy'])
# plt.title('Conv1D close world testing accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['testing accuracy'], loc='upper left')


# plt.savefig('Conv1D close world testing accuracy.png', bbox_inches='tight')
# plt.show()

# files.download('Conv1D close world testing accuracy.png')

model 3 got 88.29%

##model 4

In [ ]:
# model_4 = tf.keras.Sequential([
#   layers.Dense(5000, activation="relu"),
#   layers.Dense(750, activation="relu"),

#   layers.Lambda(lambda x: tf.expand_dims(x, axis = 1)),
#   layers.Conv1D(500, 5, padding = 'causal', activation="relu"),
#   layers.GlobalMaxPooling1D(),

#   layers.Dense(750, activation="relu"),
#   layers.Dense(5000, activation="relu"),


#   layers.Dense(100, activation="softmax")
# ], name = "Conv1D")

# model_4.compile(loss = "categorical_crossentropy",
#                 optimizer = "Adam",
#                 metrics = ["accuracy"])

# history_4 = model_4.fit(train_dataset, epochs = 40,
#                         validation_data=test_dataset, 
#                     callbacks = [make_model_checkpoint("model_4")])

##model 5

In [ ]:
# model_5 = tf.keras.models.Sequential([
#   layers.Lambda(lambda x: tf.expand_dims(x, axis = 1)),
#   layers.LSTM(5000, return_sequences=True, name="lstm_layer"),
#   layers.GlobalMaxPooling1D(),  
#   layers.Dense(100, activation="softmax")
# ], name = "LSTM-BASIC")

# model_5.compile(loss = "categorical_crossentropy",
#                 optimizer = "Adam",
#                 metrics = ["accuracy"])

# history_5 = model_5.fit(train_dataset, epochs = 40,
#                         validation_data=test_dataset, 
#                     callbacks = [make_model_checkpoint("model_5")])



In [ ]:
# new_model_5 = tf.keras.models.load_model("/content/model_exp/model_5")
# new_model_5.evaluate(test_dataset)

88.11% accuracy

#Ensemble model

In [ ]:
def make_2layer_SDAE(layer_1, center_layer, name):
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(center_layer, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, "softmax", kernel_initializer="he_normal"),
  ])
  model.compile(loss = "categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics = ["accuracy"],
              )
  model.fit(train_dataset, epochs = 100,
                        validation_data=test_dataset, 
                    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)], verbose=None)
  return model

In [ ]:
def make_3layer_SDAE(layer_1, layer_2, center_layer):
  model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_2, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(center_layer, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_2, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(layer_1, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, "softmax", kernel_initializer="he_normal"),
  ])
  model_1.compile(loss = "categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics = ["accuracy"],
              )
  model_1.fit(train_dataset, epochs = 20,
                        validation_data=test_dataset, 
                    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)], verbose=None)
  
  return model_1

In [ ]:
ensemble_models = []
#add in 2 layers

i = 1000
while i > 0:
  j = i

  while j > 0:
    print("layer 1:\t" + str(i) + "\tlayer 2:\t" + str(j))
    ensemble_models.append(make_2layer_SDAE(i, j, "model_"+str(i)+"_"+str(j)))
    j -= 250
  i-=250



In [ ]:
ensemble_models

In [ ]:
def make_ensemble_preds(ensemble, input):
  preds = []
  for model in ensemble:
    preds.append(one_hot_encoder.inverse_transform(model.predict(input)))
    
  return tf.constant(preds)

In [ ]:
x = make_ensemble_preds(ensemble_models, test_dataset)
predictions = []
for j in range(len(one_hot_test_labels)):
  ans = []
  for i in range(len(ensemble_models)):
    ans.append(x[i][j])
  a = tf.math.bincount(ans)
  a = tf.math.argmax(a).numpy()
  predictions.append(a)
  # print(a)

y_preds = np.array(predictions, dtype=int)

In [ ]:
y_true = test_df["page_num"].to_numpy().astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
accuracy = accuracy_score(y_true, y_preds)
accuracy

In [ ]:
def evaluate(y_true, y_pred):
  #model accuracy
  accuracy = accuracy_score(y_true, y_pred)

  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")

  results = {
      "accuracy": accuracy * 100,
      "precision": precision * 100,
      "recall": recall *100,
      "f1 score": f1 * 100
  }

  return results

In [ ]:
ensemble_evaluation = evaluate(y_true, y_preds)
ensemble_evaluation

In [ ]:
y_preds

In [ ]:
!nvidia-smi

In [ ]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(750, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(500, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(750, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(5000, "relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, "softmax", kernel_initializer="he_normal"),
])
model_1.compile(loss = "categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics = ["accuracy"],
              )
model_1.fit(train_dataset, epochs = 40,
                        validation_data=test_dataset, 
                    callbacks = [make_model_checkpoint("model_1")])

new_model_1 = tf.keras.models.load_model("/content/model_exp/model_1")
new_model_1.evaluate(test_dataset)

In [ ]:
layer2_preds = one_hot_encoder.inverse_transform(new_model_1.predict(test_dataset))
y_layer2_preds = np.array(layer2_preds, dtype=int)
y_layer2_preds = y_layer2_preds.reshape(1, -1).flatten()
sdae_evaluation = evaluate(y_true, y_layer2_preds)
sdae_evaluation